# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [394]:
import pandas as pd
import requests
import numpy as np

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [395]:
# 2) criando um DataFrame e contando valores missing em geral
df = pd.DataFrame(sinasc)
df.isna().sum()


ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [396]:
# 9) Renomeando as colunas para snakecase
df.rename(columns={
    'LOCNASC': 'loc_nasc',
    'IDADEMAE': 'idade_mae',
    'ESTCIVMAE': 'est_civ_mae',
    'ESCMAE': 'esc_mae',
    'QTDFILVIVO': 'qtd_fil_vivo',
    'GESTACAO': 'gestacao',
    'GRAVIDEZ': 'gravidez',
    'CONSULTAS': 'consultas',
    'APGAR5': 'apgar5'
}, inplace=True)

In [397]:
# 3) separando e contando os valores missing das colunas de interesse
colunas_interessantes = ['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo', 
    'gestacao', 'gravidez', 'consultas', 'apgar5'] 
colunas_interessantes = df[colunas_interessantes]
colunas_interessantes.isna().sum()

loc_nasc           0
idade_mae          0
est_civ_mae      317
esc_mae          312
qtd_fil_vivo    1573
gestacao        1232
gravidez          79
consultas          0
apgar5           103
dtype: int64

In [398]:
# 4) apagando os valores missing da coluna 'APGAR5'
df.dropna(subset=['apgar5'], inplace=True)
colunas_interessantes = df[colunas_interessantes.columns]
colunas_interessantes.isna().sum()


loc_nasc           0
idade_mae          0
est_civ_mae      315
esc_mae          310
qtd_fil_vivo    1566
gestacao        1216
gravidez          76
consultas          0
apgar5             0
dtype: int64

In [399]:
# 5) substituindo os valores missing por 9.0
df['est_civ_mae'].fillna(9.0, inplace=True)
df['consultas'].fillna(9.0, inplace=True)
colunas_interessantes = df[colunas_interessantes.columns]
colunas_interessantes.isna().sum()

loc_nasc           0
idade_mae          0
est_civ_mae        0
esc_mae          310
qtd_fil_vivo    1566
gestacao        1216
gravidez          76
consultas          0
apgar5             0
dtype: int64

In [400]:
# 6) Substituindo os valores missing da coluna 'QTDFILVIVO', por zero e conferindo a quantidade de valores missing
df['qtd_fil_vivo'].fillna(0, inplace=True)
colunas_interessantes = df[colunas_interessantes.columns]
colunas_interessantes.isna().sum()
print('Os valores faltantes foram substituídos por zero e a quantidade de valores faltantes é:', df['qtd_fil_vivo'].isna().sum())

Os valores faltantes foram substituídos por zero e a quantidade de valores faltantes é: 0


In [401]:
# 7) Verificando mais colunas que precisam ser alteradas e substituindo valores missing, escolaridade missing = 1 = Nenhuma, Gravidez = 1 = Única o restante está como ignorado.
df['loc_nasc'].fillna(9.0, inplace=True)
df['esc_mae'].fillna(1, inplace=True) 
df['gestacao'].fillna(9.0, inplace=True)
df['gravidez'].fillna(1, inplace=True)
colunas_interessantes = df[colunas_interessantes.columns]
colunas_interessantes.isna().sum()

loc_nasc        0
idade_mae       0
est_civ_mae     0
esc_mae         0
qtd_fil_vivo    0
gestacao        0
gravidez        0
consultas       0
apgar5          0
dtype: int64

In [408]:
# 8) classificando como 'asfixia normal', 'asfixia leve', 'asfixia moderada' e 'asfixia severa' de acordo com a nota do APGAR5
df.loc[df['apgar5'] >= 8, 'apgar5_cat'] = 'asfixia normal'
df.loc[(df['apgar5'] >= 6) & (df['apgar5'] <= 7), 'apgar5_cat'] = 'asfixia leve'
df.loc[(df['apgar5'] >= 4) & (df['apgar5'] <= 5), 'apgar5_cat'] = 'asfixia moderada'
df.loc[df['apgar5'] <= 3, 'apgar5_cat'] = 'asfixia severa'

freq_apgar5_cat = df['apgar5_cat'].value_counts()
print('De acordo com as notas do APGAR5, essas foram as quantidadesde cada uma das asfixias:\n')
print(freq_apgar5_cat)

De acordo com as notas do APGAR5, essas foram as quantidadesde cada uma das asfixias:

asfixia normal      26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: apgar5_cat, dtype: int64


In [409]:
colunas_interessantes

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0
